<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_finetuning_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## (3)Finetuning de GPT 3 para clasificación multietiqueta

**Natalia Dedandi**

Esta notebook prepara los datos para el finetuning con GPT 3.5 turbo para la clasificación multietiqueta.


La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction


Guia para finetuning:

https://medium.com/@garethcull/fine-tuning-ai-models-a-practical-guide-for-beginners-dc313b2e0f76

In [46]:
pip install openai

In [47]:
!pip install datasets seaborn


In [48]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [49]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Levanto los datos de entrenamiento

In [50]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


Tengo que transformar el DS de train en un conjunto de mensajes para pasarle a GPT para finetuning.

In [51]:
# prompt: Create a dataframe from ds1 with id text and hateful cols

df_train = pd.DataFrame(ds1['train'])[['id', 'text', 'HATEFUL','CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL']]


In [52]:
# prompt: create a datfram from ds1 for train
df_train[0:10]

,id,text,HATEFUL,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,343726,@usuario Uno menos,1,0,0,0,0,0,0,0,0,1
1,384799,@usuario #QueSeVayanTodos @usuario @usuario @u...,0,0,0,0,0,0,0,0,0,0
2,399435,"@usuario Te falta negociar con Cuba, mamerto.",0,0,0,0,0,0,0,0,0,0
3,403171,@usuario Se prepara para las milicias armadas,0,0,0,0,0,0,0,0,0,0
4,394186,@usuario Seguimos alimentando vagos,1,0,0,0,0,1,0,0,0,0
5,345041,@usuario Mmmmm,0,0,0,0,0,0,0,0,0,0
6,339268,@usuario ¿Copiando la receta de la sopa de mur...,0,0,0,0,0,0,0,0,0,0
7,360180,@usuario lastima que no murio... necesitamos u...,1,0,0,0,0,0,1,0,0,0
8,388977,@usuario Un Grande Gildo.......,0,0,0,0,0,0,0,0,0,0
9,374959,"@usuario No sé olvide Santiago Cafiero ,cuando...",0,0,0,0,0,0,0,0,0,0


In [53]:
# ajustar los valores y crear una variable binaria entera 1 y 0
df_train['CALLS_label'] = df_train['CALLS'].apply(lambda x: 'CALLS' if  x==1 else "" )
df_train['WOMEN_label'] = df_train['WOMEN'].apply(lambda x: 'WOMEN' if  x==1 else "")
df_train['LGBTI_label'] = df_train['LGBTI'].apply(lambda x: 'LGBTI' if  x==1 else "")
df_train['RACISM_label'] = df_train['RACISM'].apply(lambda x: 'RACISM' if x==1 else "" )
df_train['CLASS_label'] = df_train['CLASS'].apply(lambda x: 'CLASS' if  x==1  else "")
df_train['POLITICS_label'] = df_train['POLITICS'].apply(lambda x: 'POLITICS' if  x==1  else "")
df_train['DISABLED_label'] = df_train['DISABLED'].apply(lambda x: 'DISABLED' if  x==1  else "")
df_train['APPEARANCE_label'] = df_train['APPEARANCE'].apply(lambda x: 'APPEARANCE' if  x==1  else "")
df_train['CRIMINAL_label'] = df_train['CRIMINAL'].apply(lambda x: 'CRIMINAL' if  x==1  else "")

# Create a new column by merging non-empty "_label" columns
df_train['predicted_labels'] = df_train.filter(like='_label').apply(lambda x: ', '.join(x[x != '']), axis=1)

In [54]:

df_test = pd.DataFrame(ds1['test'])[['id', 'text', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL']]


In [55]:
df_test['CALLS_label'] = df_test['CALLS'].apply(lambda x: 'CALLS' if  x==1 else "" )
df_test['WOMEN_label'] = df_test['WOMEN'].apply(lambda x: 'WOMEN' if  x==1 else "")
df_test['LGBTI_label'] = df_test['LGBTI'].apply(lambda x: 'LGBTI' if  x==1 else "")
df_test['RACISM_label'] = df_test['RACISM'].apply(lambda x: 'RACISM' if x==1 else "" )
df_test['CLASS_label'] = df_test['CLASS'].apply(lambda x: 'CLASS' if  x==1  else "")
df_test['POLITICS_label'] = df_test['POLITICS'].apply(lambda x: 'POLITICS' if  x==1  else "")
df_test['DISABLED_label'] = df_test['DISABLED'].apply(lambda x: 'DISABLED' if  x==1  else "")
df_test['APPEARANCE_label'] = df_test['APPEARANCE'].apply(lambda x: 'APPEARANCE' if  x==1  else "")
df_test['CRIMINAL_label'] = df_test['CRIMINAL'].apply(lambda x: 'CRIMINAL' if  x==1  else "")

# Create a new column by merging non-empty "_label" columns
df_test['predicted_labels'] = df_test.filter(like='_label').apply(lambda x: ', '.join(x[x != '']), axis=1)

In [56]:
df_test[0:5]

,id,text,HATEFUL,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,...,CALLS_label,WOMEN_label,LGBTI_label,RACISM_label,CLASS_label,POLITICS_label,DISABLED_label,APPEARANCE_label,CRIMINAL_label,predicted_labels
0,397703,@usuario Ojalá se funda y cague de hambre,0,0,0,0,0,0,0,0,...,,,,,,,,,,
1,397704,@usuario Faltó que se enganchó al hijo de sofo...,1,0,1,0,0,0,0,0,...,,WOMEN,,,,,,,,WOMEN
2,397705,"@usuario y bue, que le vaya a llorar a Cristin...",0,0,0,0,0,0,0,0,...,,,,,,,,,,
3,397706,@usuario @usuario Jodete. Macri te hizo conoce...,0,0,0,0,0,0,0,0,...,,,,,,,,,,
4,397707,"@usuario Que se joda, es su gobierno el que el...",0,0,0,0,0,0,0,0,...,,,,,,,,,,


Creo una lista BATCH con el formato de mensajes que debo enviar a GPT para finetunning

In [57]:

  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row['text']},
                  {'role': 'assistant', 'content': row['predicted_labels']}
                  ]
       }
      for _, row in df_train.iterrows()
  ]


Preparo uno de train y uno de test para pasarle ambos al clasificador

In [58]:
  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch_test = [
      {'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row['text']},
                  {'role': 'assistant', 'content': row['predicted_labels']}
                  ]
       }
      for _, row in df_test.iterrows()
  ]

In [59]:
batch[0]

{'messages': [{'role': 'system',
   'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
  {'role': 'user', 'content': '@usuario Uno menos'},
  {'role': 'assistant', 'content':

In [63]:
# Save the batch to a .jsonl file
with open('batch_gpt_multi_train_v3.jsonl', 'w') as f:
    for entry in batch:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [64]:
with open('batch_gpt_multi_test_v3.jsonl', 'w') as f:
    for entry in batch_test:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


Ya tengo creado el archivo y lo subo para finetunning

In [65]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("batch_gpt_multi_train_v3.jsonl", "rb"),
  purpose="fine-tune"
)



FileObject(id='file-R82RagvEYz2teWwC9qRBx12Q', bytes=39293286, created_at=1723315489, filename='batch_gpt_multi_train_v3.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [66]:
client.files.create(
  file=open("batch_gpt_multi_test_v3.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-owHvdiZn5mxubs7bsO6ibuxp', bytes=12218588, created_at=1723315498, filename='batch_gpt_multi_test_v3.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [67]:
client.fine_tuning.jobs.create(
  training_file="file-R82RagvEYz2teWwC9qRBx12Q",
  validation_file="file-owHvdiZn5mxubs7bsO6ibuxp",
  model="gpt-3.5-turbo-0125",
  hyperparameters={
    "n_epochs":2
  },
  suffix="hatemultiv3"
)

FineTuningJob(id='ftjob-vWJxjkUshezXgbUPgqTU6Kbr', created_at=1723315922, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=921469625, status='validating_files', trained_tokens=None, training_file='file-R82RagvEYz2teWwC9qRBx12Q', validation_file='file-owHvdiZn5mxubs7bsO6ibuxp', estimated_finish=None, integrations=[], user_provided_suffix='hatemultiv3')

In [68]:
fine_tuning_job = client.fine_tuning.jobs.retrieve(
  fine_tuning_job_id="ftjob-vWJxjkUshezXgbUPgqTU6Kbr"
)

print(fine_tuning_job)


FineTuningJob(id='ftjob-vWJxjkUshezXgbUPgqTU6Kbr', created_at=1723315922, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=921469625, status='validating_files', trained_tokens=None, training_file='file-R82RagvEYz2teWwC9qRBx12Q', validation_file='file-owHvdiZn5mxubs7bsO6ibuxp', estimated_finish=None, integrations=[], user_provided_suffix='hatemultiv3')


In [69]:
fine_tuning_job.status

'validating_files'